## Install Minikube

Following command should work for Linux, for other OS, see [Minikube docs](https://github.com/kubernetes/minikube/blob/master/README.md#installation)

```
curl -LO https://storage.googleapis.com/minikube/releases/latest/minikube-linux-amd64 &&\
sudo install minikube-linux-amd64 /usr/local/bin/minikube &&\
rm minikube-linux-amd64
```

### (Optional) Ensure a clean Minikube cluster

```
minikube delete; rm -rf ~/.minikube
```

## Initialize the Odata Minikube environment

Create the host path for persistent storage

```
sudo mkdir -p /var/odata-minikube-storage
```

Switch to the Odata Minikube environment

```
source switch_environment.sh odata-minikube
```

Initialize the cluster

```
charts-external/odata/deploy.sh --install-minikube
```

## Switch to the Odata Minikube environment

In [66]:
source switch_environment.sh odata-minikube

minikube$ Switching to odata-minikube  environment
Switched to context "minikube".
Connected to minikube


: 1

## Deploy

You can either Deploy a clean environment from scratch or deploy from back (if you have the required credentials)

### Deploy from scratch

In [ ]:
charts-external/odata/deploy.sh --initialize

### Deploy from backup

You will need a Google service account file with permissions to the odata backups bucket

In [67]:
DEPLOY_FROM_BACKUP_SERVICE_ACCOUNT=/etc/ckan/odata-k8s-service-account.json
# BACKUP_GS_URL="gs://odata-k8s-backups/production/blue/ckan-db-dump-$(date +%Y-%m-%d).ckan.dump"
BACKUP_GS_URL="gs://odata-k8s-backups/production/blue/ckan-db-dump-2018-10-10.ckan.dump"

charts-external/odata/deploy.sh --restore "${DEPLOY_FROM_BACKUP_SERVICE_ACCOUNT}" "${BACKUP_GS_URL}"

minikube$ minikube$ minikube$ minikube$ minikube$ Creating DB restore secret
secret "ckan-db-restore" created
Switched to context "minikube".
Connected to minikube
RELEASE_NAME=odata-minikube-odata-odata-minikube
CHART_DIRECTORY=charts-external/odata
Release "odata-minikube-odata-odata-minikube" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 11 21:57:45 2018
NAMESPACE: odata-minikube
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME   AGE
ckan   1d
nginx  1d

==> v1/Service
ckan-jobs-db  1d
ckan          1d
datapusher    1d
datastore-db  1d
db            1d
nginx         1d
redis         1d
solr          1d

==> v1beta1/Deployment
ckan          4s
datastore-db  1d
db            1d
nginx         4s
redis         1d
solr          1d

==> v1/Pod(related)

NAME                          READY  STATUS             RESTARTS  AGE
ckan-8cd5cf798-w6pxw          0/1    Init:0/3           0         3s
datastore-db-5665c8bc5-6ljkk  1/1    Terminating        0         2h
db-84d9d5477b-284l

: 1

### Wait for pods to be in Running state

In [77]:
kubectl get pods

minikube$ NAME                           READY     STATUS    RESTARTS   AGE
ckan-8cd5cf798-w6pxw           1/1       Running   0          8m
datastore-db-5db4c9588-n69nk   1/1       Running   1          8m
db-5cdddcbdb8-rpc2g            2/2       Running   0          8m
nginx-69d85f565f-dkfpm         1/1       Running   0          8m
nginx-69d85f565f-z66hp         1/1       Running   0          8m
redis-6ff99b65d-sz4gm          1/1       Running   0          2h
solr-5d586f64db-rkhdf          1/1       Running   0          8m


: 1

Some pods might take a while on first deploy due to pulling large Docker images and waiting for other components, please be patient.. it will start eventually

You can use the following commands to inspect the main pods (ckan / datastore-db / db etc..)

In [ ]:
# Describe the first pod serving deployment ckan
./kubectl.sh describe ckan

# Get logs of container secrets in the first pod serving deployment ckan
./kubectl.sh logs ckan -c secrets

# Get yaml of first pod serving deployment ckan
./kubectl.sh get ckan -o yaml

If CKAN API responds it's a good indication that environment was setup successfully

In [78]:
echo $(./kubectl.sh exec ckan -- wget -qO - http://127.0.0.1:5000/api/3)

minikube$ {"version": 3}


: 1

It should respond with `{"version": 3}`

## Start port forwarding to access the CKAN web-app

Open a new terminal and run:

```
./kubectl.sh port-forward nginx 5000:80
```

You will need to restart it if CKAN pod is restarted

The web app should be available at http://localhost:5000

## Start the pipelines server

In [131]:
source switch_environment.sh odata-minikube
./helm_upgrade_external_chart.sh odata --force \
    --set pipelines.enabled=true \
    --set pipelines.sysadminUser=minikube-pipelines-sysadmin-`date +%s` \
    --set pipelines.sysadminEmail=minikube-pipelines-sysadmin-`date +%s`@localhost \
    --set pipelines.secretName=minikube-pipelines-sysadmin-`date +%s`

minikube$ Switching to odata-minikube  environment
Switched to context "minikube".
Connected to minikube
minikube$ Error from server (NotFound): secrets "minikube-pipelines-sysadmin" not found
minikube$ Switched to context "minikube".
Connected to minikube
RELEASE_NAME=odata-minikube-odata-odata-minikube
CHART_DIRECTORY=charts-external/odata
Release "odata-minikube-odata-odata-minikube" has been upgraded. Happy Helming!
LAST DEPLOYED: Thu Oct 11 22:23:53 2018
NAMESPACE: odata-minikube
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME   AGE
ckan   1d
nginx  1d

==> v1/Service
ckan-jobs-db  1d
ckan          1d
datapusher    1d
datastore-db  1d
db            1d
nginx         1d
pipelines     17m
redis         1d
solr          1d

==> v1beta1/Deployment
ckan          26m
ckan-jobs-db  17m
ckan-jobs     1m
datastore-db  1d
db            1d
nginx         26m
pipelines     17m
redis         1d
solr          1d

==> v1/Pod(related)

NAME                           READY  STATUS       RESTARTS 

: 1

#### Wait for pods to be in Running state

In [154]:
kubectl get pods

minikube$ NAME                            READY     STATUS    RESTARTS   AGE
ckan-745967dbb5-4m79p           1/1       Running   1          14m
ckan-jobs-d96889856-29cwp       2/2       Running   0          3m
ckan-jobs-db-54565fbc7c-tx9mv   1/1       Running   0          19m
datastore-db-5665c8bc5-7tc9m    1/1       Running   0          18m
db-84d9d5477b-mxhtw             1/1       Running   0          18m
nginx-69d85f565f-dkfpm          1/1       Running   0          28m
nginx-69d85f565f-z66hp          1/1       Running   0          28m
pipelines-785b68d89d-lfb4d      1/1       Running   0          1m
redis-6ff99b65d-sz4gm           1/1       Running   0          2h
solr-76b79ddfdb-fk25s           1/1       Running   0          17m


: 1

#### Using the pipelines server

In Minikube environment pipelines server is run in manual mode by default, so the pipelines dashboard is not available and scheduled pipelines are disabled.

You can run the pipelines manually

Pipelines server might take a minute to start, if you encounter problems, try to restart it and also the ckan server which updates the pipelines code on startup.

#### List the available pipelines

In [155]:
./kubectl.sh exec pipelines dpp

minikube$ Available Pipelines:
- ./ckanext-upload_via_email/upload_via_email (*)
- ./ckanext-odata_org_il/download_foi_offices 
- ./ckanext-odata_org_il/download_foi_offices_xml (*)
- ./ckanext-odata_org_il/download_foi_offices_matching_resource (*)
- ./ckanext-odata_org_il/update_foi_offices_resource (*)
- ./ckanext-odata_org_il/update_foi_offices_entities (*)(E)
	Dirty dependency: Cannot run until dependency is executed: ./ckanext-odata_org_il/update_foi_offices_resource
	Dirty dependency: Cannot run until dependency is executed: ./ckanext-odata_org_il/download_foi_offices_matching_resource
- ./ckanext-odata_org_il/export_entities_resource (*)
- ./ckanext-odata_org_il/dump_group_datasets 


: 1

#### Run a pipeline

In [ ]:
./kubectl.sh exec pipelines -- dpp run --verbose ./ckanext-odata_org_il/download_foi_offices

#### Check the generated datapackage

In [ ]:
./kubectl.sh exec pipelines -- cat /var/lib/ckan/data/pipelines/odata_org_il/new_foi_offices/datapackage.json | jq

#### Additional Tasks

In [159]:
./kubectl.sh exec pipelines -- dpp init &&\
./kubectl.sh exec pipelines -- dpp run ./ckanext-odata_org_il/dump_group_datasets

minikube$ INFO    :RESULTS:
INFO    :SUCCESS: ./ckanext-odata_org_il/dump_group_datasets {'bytes': 188467, 'count_of_rows': 414, 'dataset_name': '_', 'existing_groups_without_entity': 207, 'hash': '3191e5d0642d71392f68c829c18bc31f'}

./ckanext-odata_org_il/dump_group_datasets: WAITING FOR OUTPUT

./ckanext-odata_org_il/dump_group_datasets: RUNNING, processed 100 rows

./ckanext-odata_org_il/dump_group_datasets: RUNNING, processed 200 rows

./ckanext-odata_org_il/dump_group_datasets: RUNNING, processed 300 rows

./ckanext-odata_org_il/dump_group_datasets: RUNNING, processed 400 rows

./ckanext-odata_org_il/dump_group_datasets: RUNNING, processed 414 rows

./ckanext-odata_org_il/dump_group_datasets: SUCCESS, processed 414 rows


: 1

In [168]:
DATASET_ID="8f9244c5-2cef-4f71-bc28-d73da1ed6368"
GROUP_ID=`kubectl exec $(./kubectl.sh get-pod-name pipelines) -- bash -c \
              'cat /var/lib/ckan/data/pipelines/odata_org_il/dump_group_datasets/group_datasets.csv | grep '${DATASET_ID}' | cut -d"," -f1 -'`

echo
echo DATASET_ID=${DATASET_ID}
echo GROUP_ID=${GROUP_ID}

kubectl exec $(./kubectl.sh get-pod-name pipelines) -- bash -c \
    'cat /var/lib/ckan/data/pipelines/odata_org_il/dump_group_datasets/existing_entities.csv | grep '${GROUP_ID}

minikube$ minikube$ minikube$ minikube$ 
minikube$ DATASET_ID=8f9244c5-2cef-4f71-bc28-d73da1ed6368
minikube$ GROUP_ID=4ae1ad21-c186-425f-8f5a-6c75f9eee1ae
minikube$ minikube$ אוניברסיטת אריאל בשומרון,4ae1ad21-c186-425f-8f5a-6c75f9eee1ae,ariel,,"{""approval_status"": ""approved"", ""created"": ""2018-04-09T09:00:55.264550"", ""description"": """", ""display_name"": ""אוניברסיטת אריאל בשומרון"", ""extras"": [{""group_id"": ""4ae1ad21-c186-425f-8f5a-6c75f9eee1ae"", ""id"": ""72794b42-487a-4c78-b329-ee719efd5986"", ""key"": ""OfficeNameCode"", ""revision_id"": ""fd94e3ec-7d01-4291-983d-f38dbf5fa8fc"", ""state"": ""active"", ""value"": ""7047""}], ""id"": ""4ae1ad21-c186-425f-8f5a-6c75f9eee1ae"", ""image_display_url"": ""http://www.ariel.ac.il/templates/ArielUniversityHEv1/images/new_logo.png"", ""image_url"": ""http://www.ariel.ac.il/templates/ArielUniversityHEv1/images/new_logo.png"", ""is_organization"": false, ""name"": ""ariel"", ""num_followers"": 0, ""revision_id"": ""ec2a303f-e9dc-4

: 1

In [ ]:
source switch_environment.sh odata
# kubectl get pods
git pull hasadna master
./helm_upgrade_external_chart.sh odata
# kubectl exec $(./kubectl.sh get-pod-name pipelines) -- dpp

hasadna-general-hasadna-cluster-odata-blue$ Switching to odata  environment
Switched to context "gke_hasadna-general_europe-west1-b_hasadna-cluster".
Connected to hasadna-general-hasadna-cluster-odata-blue
hasadna-general-hasadna-cluster-odata-blue$ 

## Local Development of odata ckan

#### Install supervisord

Supervisord is used to run and manage background processes for the development environment

```
sudo easy_install supervisor
```

#### Ensure all components are deployed and working


In [ ]:
source switch_environment.sh odata-minikube
./helm_upgrade_external_chart.sh odata --force --set pipelines.enabled=true

In [ ]:
kubectl get pods

#### Start the supervisord daemon

This starts port forwards for all the infrastructure componnets from localhost to the Minikube cluster

In [ ]:
supervisord -c environments/odata-minikube/local-dev-supervisord.conf

#### Ensure all port forwards are running

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status

#### Try a reload if there are problems

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf reload

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status

#### Clone the data4dappl project

In [ ]:
! [ -e ../data4dappl ] && git clone https://github.com/OriHoch/data4dappl.git ../data4dappl

#### Prepare the CKAN configuration file

This copies the configuartion from the minikube ckan pod and modifies some values to make it suitable for local development

In [ ]:
CKAN_LOCAL_DEVELOPMENT_INI="environments/odata-minikube/ckan-local-development.ini"
CKAN_LOCAL_DEVELOPMENT_WHO_INI="environments/odata-minikube/who.ini"

# delete the existing file to force recreate of the configuration file
rm -f "${CKAN_LOCAL_DEVELOPMENT_INI}"

if ! [ -e "${CKAN_LOCAL_DEVELOPMENT_INI}" ]; then
    echo Modfying configuration file from pod, saving to "${CKAN_LOCAL_DEVELOPMENT_INI}"
    ./kubectl.sh exec ckan cat /etc/ckan/production.ini \
        | sed -e 's/redis:6379/localhost:6379/g' \
        | sed -e 's/solr:8983/localhost:8983/g' \
        | sed -e 's/db\/ckan/localhost\/ckan/g' \
        | sed -e 's/ckan-jobs-db\/postgres/localhost:5433\/postgres/g' \
        | sed -e 's/datastore-db\/datastore/localhost:5434\/datastore/g' \
            > "${CKAN_LOCAL_DEVELOPMENT_INI}" &&\
    ./kubectl.sh exec ckan cat /etc/ckan/who.ini > "${CKAN_LOCAL_DEVELOPMENT_WHO_INI}"
    if [ "$?" == "0" ]; then
        echo Great Success
    else
        echo Failed to process configuration file
    fi
else
    echo Configuration file already exists, delete to recreate: "${CKAN_LOCAL_DEVELOPMENT_INI}"
fi

#### Install the project dependencies

In [ ]:
( cd ../data4dappl; bin/install.sh )

#### Start Gunicorn which serves the CKAN web-app

In [ ]:
( cd ../data4dappl
  pipenv run gunicorn \
      --paste ../hasadna-k8s/${CKAN_LOCAL_DEVELOPMENT_INI:-environments/odata-minikube/ckan-local-development.ini} )

Site should be available at http://localhost:5000

#### Start Gunicorn in the background

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf start ckan-gunicorn

#### Get Gunicorn logs

In [ ]:
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf tail ckan-gunicorn

#### Restart Services and port-forwards

Once everything is setup you should run this to ensure services are connected and working properly

Occasionally you might need to run this again if you encounter wierd problems

In [ ]:
echo Remove deployments which should run locally
kubectl delete deployment --namespace odata-minikube --context minikube ckan nginx ckan-jobs pipelines

echo Force restart of infrastructure components
for DEPLOYMENT in db datastore-db ckan-jobs-db redis solr; do
    ./force_update.sh $DEPLOYMENT &
done
wait

echo reload gunicorn
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf reload
while [ "5" != "`supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status | tee -a /dev/stderr | grep RUNNING | wc -l`" ]
do sleep 2; done

echo Start all services
supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf start all
while [ "7" != "`supervisorctl -c environments/odata-minikube/local-dev-supervisord.conf status | tee -a /dev/stderr | grep RUNNING | wc -l`" ]
do sleep 2; done

Site should be available at http://localhost:5000

## Local Development of Odata CKAN Pipelines

You should run the steps for local development of CKAN first

Create a pipenv for the pipelines

In [ ]:
TEMPFILE=`mktemp`
curl https://raw.githubusercontent.com/OriHoch/ckanext-datapackage_pipelines/master/datapackage_pipelines_ckanext/requirements.txt > $TEMPFILE
( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv install --python 3 &&\
  pipenv install -r $TEMPFILE &&\
  pipenv run pip install \
      -e 'git+https://github.com/OriHoch/ckanext-datapackage_pipelines#subdirectory=datapackage_pipelines_ckanext&egg=datapackage_pipelines_ckanext'
)
rm $TEMPFILE

#### Set the required env vars to run the pipelines

If you are restoring from backup, you can use your own admin api key which you can get from the web UI

In [ ]:
export CKAN_API_KEY=`kubectl get secrets --context=minikube --namespace=odata-minikube pipelines-sysadmin -o json | jq -r .data.apikey | base64 -d`
export CKAN_URL='http://localhost:5000'

#### List the available pipelines

In [ ]:
( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv run dpp )

#### Run a pipeline

In [ ]:
# Force recreate of the foi offices resource
rm -rf /var/lib/ckan/pipelines/odata_org_il/foi_offices

( cd ../data4dappl/ckan/ckanext-odata_org_il/ckanext/odata_org_il/pipelines
  pipenv run dpp run --verbose ./download_foi_offices,./update_foi_offices_resource )